### 4E1

y ~ Normal(mu, sigma)

### 4E2

Two parameters

### 4E3

$y_i \sim \rm{Normal}(\mu, \sigma)$

$\mu \sim \rm{Normal}(0, 10)$

$\sigma \sim \rm{Exponential}(1)$

$P(y | \mu{}, \sigma{})=\frac{\rm{Normal}(y | \mu{}, \sigma{})\rm{Normal}(\mu{} | 0, 10)\rm{Exponential}(\sigma | 1)}{\int{\rm{Normal}(y | \mu{}, \sigma{})\rm{Normal}(\mu{} | 0, 10)\rm{Exponential}(\sigma | 1)}dy}$

### 4E4

Second line

### 4E5

Four parameters ($\mu{}, \alpha{}, \beta{}, \sigma{}$)

In [ ]:
# 4M1

